<a href="https://colab.research.google.com/github/cselester/ShaadiVerse/blob/main/ShaadiVerse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU gradio langchain langchain-groq fpdf2

import gradio as gr
from langchain_groq import ChatGroq
from google.colab import userdata
from fpdf import FPDF
import requests, os, json

# ===================== 🔑 Setup Groq API =====================
groq_api_key = userdata.get('GROQ_API_KEY')

llm = ChatGroq(
    temperature=0.4,
    groq_api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile"
)

# ===================== 📦 Data Persistence for Multi-Agent Flow =====================
SESSION_FILE = "wedding_session.json"

def save_session(data):
    """Save the collected wedding planning inputs for next agent."""
    with open(SESSION_FILE, "w") as f:
        json.dump(data, f)
    return True

def load_session():
    """Load stored wedding data (if exists)."""
    if os.path.exists(SESSION_FILE):
        with open(SESSION_FILE, "r") as f:
            return json.load(f)
    return {}

# ===================== 🧠 Core Planning Logic =====================
def plan_with_groq(name, religion, start_date, end_date, budget, location, ceremonies, guests, clothing):
    # Store input for next agent
    session_data = {
        "name": name,
        "religion": religion,
        "start_date": start_date,
        "end_date": end_date,
        "budget": budget,
        "location": location,
        "ceremonies": ceremonies,
        "guests": guests,
        "clothing":clothing
    }
    save_session(session_data)

    prompt = f"""
    You are an AI Wedding Planner for ShaadiVerse.

    Couple Name: {name}
    Religion: {religion}
    Wedding Dates: {start_date} to {end_date}
    Ceremonies: {ceremonies}

    Task:
    - Create a complete wedding event schedule.
    - Include date, time, and brief notes for each event.
    - Ensure realistic gaps between events if us if user input.
    - Optimize sequence (e.g., Haldi → Mehendi → Sangeet → Wedding → Reception).
    - Follow Indian wedding customs for the selected religion.
    -
    Output format:
    1. [Ceremony Name] – [Date] at [Time]
       Notes: [Short description]
    """

    response = llm.invoke(prompt)
    return response.content.strip()

def update_schedule(existing_schedule, update_request):
    prompt = f"""
    Here is the current wedding schedule:

    {existing_schedule}

    User request: {update_request}

    Update the schedule accordingly.
    Maintain the same format and ensure timing/logical flow makes sense.
    """
    response = llm.invoke(prompt)
    return response.content.strip()

# ===================== 🧾 PDF Export (No Emojis) =====================
font_url = "https://github.com/dejavu-fonts/dejavu-fonts/raw/master/ttf/DejaVuSans.ttf"
font_path = "DejaVuSans.ttf"
if not os.path.exists(font_path):
    with open(font_path, "wb") as f:
        f.write(requests.get(font_url).content)

def export_pdf(schedule_text):
    pdf = FPDF()
    pdf.add_page()
    pdf.add_font("DejaVu", "", font_path, uni=True)
    pdf.set_font("DejaVu", "B", 16)
    pdf.cell(200, 10, txt="Wedding Schedule", ln=True, align="C")
    pdf.set_font("DejaVu", size=12)
    pdf.multi_cell(0, 10, txt=schedule_text)
    pdf_output_path = "wedding_schedule.pdf"
    pdf.output(pdf_output_path)
    return pdf_output_path

def finalize_pdf(schedule_text):
    pdf_path = export_pdf(schedule_text)
    return pdf_path

# ===================== 🎨 Gradio Interface =====================
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## ShaadiVerse AI Wedding Planner (Groq-Powered Smart Scheduler)")

    with gr.Row():
        name = gr.Textbox(label="Couple Name", placeholder="e.g. Aditi & Raj")
        religion = gr.Dropdown(["Hindu", "Sikh", "Muslim", "Christian", "Other"], label="Religion", value="Hindu")
    with gr.Row():
        start_date = gr.Textbox(label="Start Date (YYYY-MM-DD)", placeholder="e.g. 2026-05-25")
        end_date = gr.Textbox(label="End Date (YYYY-MM-DD)", placeholder="e.g. 2026-05-30")
    with gr.Row():
        budget = gr.Textbox(label="Budget (₹)", value="1000000")
        clothing=gr.Textbox(label="Include clothing in budget " ,placeholder="true/false" ,value="false")
        guests = gr.Number(label="Number of Guests", value=300)
    location = gr.Textbox(label="Location (City/State)", placeholder="e.g. Jaipur, Rajasthan")
    ceremonies = gr.Textbox(label="Preferred Ceremonies (comma-separated)", value="Haldi, Mehendi, Sangeet, Wedding, Reception")

    generate_btn = gr.Button("Generate AI-Optimized Schedule")
    schedule_output = gr.Textbox(label="Wedding Schedule", lines=15)

    update_box = gr.Textbox(label="Request Change (e.g., Add Reception or Move Haldi to 10 AM)", lines=3)
    update_btn = gr.Button("Update Schedule")

    finalize_btn = gr.Button("Finalize & Export PDF")
    pdf_output = gr.File(label="Download Your Wedding Schedule PDF")

    # Button Actions
    generate_btn.click(plan_with_groq, [name, religion, start_date, end_date, budget, location, ceremonies, guests, clothing], schedule_output)
    update_btn.click(update_schedule, [schedule_output, update_box], schedule_output)
    finalize_btn.click(finalize_pdf, [schedule_output], pdf_output)

demo.launch(debug=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 16.1 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can s

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e381b10e2aa1decf8b.gradio.live


In [ ]:
# Single runnable block for Google Colab — auto-fills from wedding_session.json
!pip install -qU gradio pandas matplotlib fpdf2

import os, json, math, requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from fpdf import FPDF
import gradio as gr

# ---------- Paths ----------
VENUE_CSV = "/content/Venue - Sheet1.csv"
CATERERS_CSV = "/content/Caterers - Sheet1 (2).csv"
PHOTOGRAPHER_CSV = "/content/Photographer - Sheet1 (1).csv"
MUSIC_CSV = "/content/Music Band - Sheet1 (1).csv"
DECORATORS_CSV = "/content/Decorators - Sheet1 (1).csv"

WEDDING_SESSION_FILE = "/content/wedding_session.json"   # produced by Planner Agent
BUDGET_SESSION_FILE = "/content/budget_session.json"     # produced by this Budget Agent
PDF_FONT_URL = "https://raw.githubusercontent.com/dejavu-fonts/dejavu-fonts/master/ttf/DejaVuSans.ttf"
PDF_FONT_PATH = "/content/DejaVuSans.ttf"

# ---------- Ensure font for PDF ----------
if not os.path.exists(PDF_FONT_PATH):
    try:
        r = requests.get(PDF_FONT_URL, timeout=30)
        with open(PDF_FONT_PATH, "wb") as f:
            f.write(r.content)
    except Exception as e:
        print("Font download failed:", e)

# ---------- CSV loader ----------
def safe_load_csv(path):
    if not os.path.exists(path):
        return None
    try:
        df = pd.read_csv(path)
        df.columns = [c.strip() for c in df.columns]
        return df
    except Exception as e:
        print(f"Failed reading {path}: {e}")
        return None

venues_df = safe_load_csv(VENUE_CSV)
caterers_df = safe_load_csv(CATERERS_CSV)
photographers_df = safe_load_csv(PHOTOGRAPHER_CSV)
music_df = safe_load_csv(MUSIC_CSV)
decorators_df = safe_load_csv(DECORATORS_CSV)

# ---------- Load planner session (auto-fill) ----------
def load_wedding_session():
    if not os.path.exists(WEDDING_SESSION_FILE):
        return {}
    try:
        with open(WEDDING_SESSION_FILE, "r") as f:
            data = json.load(f)
        # normalize keys and types
        # ensure numeric types
        if "budget" in data:
            try:
                data["budget"] = float(data["budget"])
            except:
                pass
        if "guests" in data:
            try:
                data["guests"] = int(data["guests"])
            except:
                pass
        # boolean clothing key normalize
        if "clothing" in data:
            val = data["clothing"]
            if isinstance(val, str):
                data["clothing"] = val.lower() in ("true","1","yes")
            else:
                data["clothing"] = bool(val)
        return data
    except Exception as e:
        print("Failed loading wedding_session.json:", e)
        return {}

planner_session = load_wedding_session()
# defaults (if not present in planner_session)
DEFAULTS = {
    "name": planner_session.get("name", ""),
    "location": planner_session.get("location", planner_session.get("city", "Lucknow")),
    "start_date": planner_session.get("start_date", ""),
    "end_date": planner_session.get("end_date", ""),
    "total_budget": planner_session.get("budget", 1500000),
    "guests": planner_session.get("guests", 300),
    "ceremonies": planner_session.get("ceremonies", "Haldi, Mehendi, Sangeet, Wedding, Reception"),
    "include_clothing": planner_session.get("clothing", True)
}

# ---------- Allocation logic ----------
def allocate_budget(total_budget, guests, num_days, include_clothing=True):
    total = float(total_budget)
    base = {
        "Venue": 0.30,
        "Catering": 0.35,
        "Photographer": 0.10,
        "Music Band": 0.08,
        "Decorator": 0.07,
        "Clothing": 0.10,
    }
    if not include_clothing:
        clothing_share = base.pop("Clothing")
        base["Catering"] += clothing_share * 0.5
        remaining = clothing_share * 0.5
        keys = ["Venue", "Photographer", "Music Band", "Decorator"]
        s = sum(base[k] for k in keys)
        for k in keys:
            base[k] += remaining * (base[k] / s)

    alloc = {k: round(v * total) for k, v in base.items()}
    diff = int(total) - sum(alloc.values())
    if diff != 0:
        alloc["Catering"] = alloc.get("Catering", 0) + diff

    if num_days <= 0: num_days = 1
    alloc_meta = {
        "allocations": alloc,
        "venue_per_day_target": round(alloc.get("Venue", 0) / num_days),
        "catering_per_plate_target": round(alloc.get("Catering", 0) / (guests + 100))
    }
    return alloc_meta

# ---------- Vendor recommendation helpers ----------
def nearest_by_price(df, price_col_name, target_price, top_n=3):
    if df is None or price_col_name not in df.columns:
        return pd.DataFrame()
    df2 = df.copy()
    df2[price_col_name] = pd.to_numeric(df2[price_col_name], errors='coerce').fillna(np.inf)
    df2["diff"] = (df2[price_col_name] - target_price).abs()
    df2 = df2.sort_values("diff").head(top_n)
    return df2.drop(columns=["diff"])

def recommend_vendors(alloc_meta, num_days, guests, location):
    """
    Use alloc_meta to pick nearest vendors for each category.
    If location != Lucknow, gracefully inform user.
    """
    if not location or location.strip().lower() != "lucknow":
        print(f"[INFO] Vendor data is currently available only for Lucknow. Got '{location}'.")
        empty_df = pd.DataFrame([{"Note": "Vendor recommendations are currently available only for Lucknow."}])
        return {
            "Venues": empty_df,
            "Caterers": empty_df,
            "Photographers": empty_df,
            "Music Bands": empty_df,
            "Decorators": empty_df
        }

    rec = {}
    v_target = alloc_meta["venue_per_day_target"]
    rec["Venues"] = nearest_by_price(venues_df, "Price", v_target, top_n=3)

    c_target = alloc_meta["catering_per_plate_target"]
    rec["Caterers"] = nearest_by_price(caterers_df, "Price", c_target, top_n=5)

    p_target = alloc_meta["allocations"].get("Photographer", 0)
    rec["Photographers"] = nearest_by_price(photographers_df, "Price", p_target, top_n=3)

    m_target = alloc_meta["allocations"].get("Music Band", 0)
    rec["Music Bands"] = nearest_by_price(music_df, "Price", m_target, top_n=3)

    d_target = alloc_meta["allocations"].get("Decorator", 0)
    rec["Decorators"] = nearest_by_price(decorators_df, "Price", d_target, top_n=3)

    return rec


# ---------- Save session for next agent ----------
def save_budget_session(session_data, alloc_meta, rec):
    payload = {
        "planner_input": session_data,
        "alloc_meta": alloc_meta,
        "recommendations_preview": {}
    }
    for k, df in rec.items():
        if df is None or df.empty:
            payload["recommendations_preview"][k] = []
        else:
            items = []
            price_col = [c for c in df.columns if c.strip().lower()=="price"]
            name_col = [c for c in df.columns if "Name" in c or "name" in c]
            for _, row in df.head(3).iterrows():
                name_val = row[name_col[0]] if name_col else str(row.iloc[0])
                price_val = row[price_col[0]] if price_col else None
                items.append({"name": str(name_val), "price": float(price_val) if price_val is not None else None})
            payload["recommendations_preview"][k] = items
    with open(BUDGET_SESSION_FILE, "w") as f:
        json.dump(payload, f, indent=2)
    # also update wedding_session.json to include latest allocations
    try:
        with open(WEDDING_SESSION_FILE, "w") as f:
            json.dump(session_data, f, indent=2)
    except Exception:
        pass
    return BUDGET_SESSION_FILE

# ---------- Visualization ----------
def plot_allocation_pie(alloc_meta):
    labels = list(alloc_meta["allocations"].keys())
    sizes = list(alloc_meta["allocations"].values())
    fig, ax = plt.subplots(figsize=(6,6))
    ax.pie(sizes, labels=labels, autopct=lambda p: f"₹{int(round(p/100*sum(sizes)))}", startangle=140)
    ax.axis('equal')
    plt.tight_layout()
    return fig

# ---------- Main function ----------
def optimize_budget_and_recommend_autofill(name, start_date, end_date, total_budget, guests, location, ceremonies, include_clothing):
    # parse dates
    try:
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.strptime(end_date, "%Y-%m-%d")
        num_days = (end_dt - start_dt).days + 1
        if num_days <= 0: num_days = 1
    except Exception:
        num_days = 1

    guests = int(guests)
    total_budget = float(total_budget)

    session_data = {
        "name": name,
        "start_date": start_date,
        "end_date": end_date,
        "location": location,
        "guests": guests,
        "budget": total_budget,
        "ceremonies": ceremonies,
        "include_clothing": bool(include_clothing),
        "num_days": num_days,
        "saved_at": datetime.utcnow().isoformat()
    }

    alloc_meta = allocate_budget(total_budget, guests, num_days, include_clothing)
    rec = recommend_vendors(alloc_meta, num_days, guests, location)

    session_file_path = save_budget_session(session_data, alloc_meta, rec)

    alloc_df = pd.DataFrame(list(alloc_meta["allocations"].items()), columns=["Category", "Allocated (₹)"])
    top_picks = {}
    for k, df in rec.items():
        if df is None or df.empty:
            top_picks[k] = pd.DataFrame([{"Note":"No data found"}])
        else:
            top_picks[k] = df.reset_index(drop=True)

    fig = plot_allocation_pie(alloc_meta)

    return alloc_df, top_picks["Venues"], top_picks["Caterers"], top_picks["Photographers"], top_picks["Music Bands"], top_picks["Decorators"], fig, session_file_path

# ---------- Gradio UI (prefilled from wedding_session.json) ----------
# Convert defaults to types Gradio expects
default_name = DEFAULTS["name"]
default_location = DEFAULTS["location"]
default_start_date = DEFAULTS["start_date"]
default_end_date = DEFAULTS["end_date"]
default_budget = DEFAULTS["total_budget"]
default_guests = DEFAULTS["guests"]
default_ceremonies = DEFAULTS["ceremonies"]
default_include_clothing = DEFAULTS["include_clothing"]

with gr.Blocks() as demo:
    gr.Markdown("## ShaadiVerse — Budget Optimizer (Auto-filled from Planner Agent)")

    with gr.Row():
        name = gr.Textbox(label="Couple Name", value=default_name)
        location = gr.Textbox(label="Location (city)", value=default_location)
    with gr.Row():
        start_date = gr.Textbox(label="Start Date (YYYY-MM-DD)", value=default_start_date)
        end_date = gr.Textbox(label="End Date (YYYY-MM-DD)", value=default_end_date)
    with gr.Row():
        total_budget = gr.Number(label="Total Budget (INR)", value=default_budget)
        guests = gr.Number(label="Expected number of guests", value=default_guests)
    ceremonies = gr.Textbox(label="Ceremonies (comma-separated)", value=default_ceremonies)
    include_clothing = gr.Checkbox(label="Include Clothing Budget?", value=default_include_clothing)

    run_btn = gr.Button("Optimize Budget & Recommend Vendors")

    alloc_table = gr.Dataframe(headers=["Category","Allocated (₹)"], label="Budget Allocation")
    venues_table = gr.Dataframe(label="Venue recommendations (closest to per-day target)")
    caterers_table = gr.Dataframe(label="Caterer recommendations (closest to per-plate target)")
    photog_table = gr.Dataframe(label="Photographer recommendations")
    music_table = gr.Dataframe(label="Music Band recommendations")
    decor_table = gr.Dataframe(label="Decorator recommendations")
    chart = gr.Plot()
    session_path_out = gr.Textbox(label="Saved session file (for next agent)")

    run_btn.click(optimize_budget_and_recommend_autofill,
                  inputs=[name, start_date, end_date, total_budget, guests, location, ceremonies, include_clothing],
                  outputs=[alloc_table, venues_table, caterers_table, photog_table, music_table, decor_table, chart, session_path_out])

demo.launch(debug=True)


test

In [12]:
# ===========================
# ShaadiVerse — Planner + Budget Dashboard (No PDF / No Cleaning)
# ===========================
!pip install -qU gradio langchain-groq pandas matplotlib

import os, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timezone
import gradio as gr
from langchain_groq import ChatGroq
from google.colab import userdata

# -------------------------
# File paths
# -------------------------
VENUE_CSV = "/content/Venue - Sheet1.csv"
CATERERS_CSV = "/content/Caterers - Sheet1 (2).csv"
PHOTOGRAPHER_CSV = "/content/Photographer - Sheet1 (1).csv"
MUSIC_CSV = "/content/Music Band - Sheet1 (1).csv"
DECORATORS_CSV = "/content/Decorators - Sheet1 (1).csv"
WEDDING_SESSION_FILE = "/content/wedding_session.json"

# -------------------------
# Load data
# -------------------------
def safe_load_csv(path):
    if not os.path.exists(path):
        return pd.DataFrame()
    return pd.read_csv(path)

venues_df = safe_load_csv(VENUE_CSV)
caterers_df = safe_load_csv(CATERERS_CSV)
photographers_df = safe_load_csv(PHOTOGRAPHER_CSV)
music_df = safe_load_csv(MUSIC_CSV)
decorators_df = safe_load_csv(DECORATORS_CSV)

# -------------------------
# LLM Setup (Groq)
# -------------------------
groq_api_key = userdata.get("GROQ_API_KEY")
llm = ChatGroq(temperature=0.4, groq_api_key=groq_api_key, model_name="llama-3.3-70b-versatile")

# -------------------------
# Load / Save session
# -------------------------
def save_wedding_session(data):
    data_copy = data.copy()
    data_copy["saved_at"] = datetime.now(timezone.utc).isoformat()
    with open(WEDDING_SESSION_FILE, "w") as f:
        json.dump(data_copy, f, indent=2)

def load_wedding_session():
    if not os.path.exists(WEDDING_SESSION_FILE):
        return {}
    try:
        with open(WEDDING_SESSION_FILE, "r") as f:
            data = json.load(f)
        if "budget" in data:
            data["budget"] = float(data["budget"])
        if "guests" in data:
            data["guests"] = int(data["guests"])
        if "clothing" in data:
            val = data["clothing"]
            data["clothing"] = val if isinstance(val, bool) else str(val).lower() in ("true","1","yes")
        return data
    except Exception:
        return {}

# -------------------------
# Planner Agent
# -------------------------
def plan_with_groq(name, religion, start_date, end_date, budget, location, ceremonies, guests, clothing):
    session = {
        "name": name, "religion": religion, "start_date": start_date,
        "end_date": end_date, "budget": budget, "location": location,
        "ceremonies": ceremonies, "guests": guests, "clothing": clothing
    }
    save_wedding_session(session)
    prompt = f"""
    You are an AI Wedding Planner for ShaadiVerse.

    Couple Name: {name}
    Religion: {religion}
    Wedding Dates: {start_date} to {end_date}
    Ceremonies: {ceremonies}

    Task:
    - Create a complete wedding event schedule.
    - Include date, time, and brief notes for each event.
    - Ensure realistic gaps between events if us if user input.
    - Optimize sequence (e.g., Haldi → Mehendi → Sangeet → Wedding → Reception).
    - Follow Indian wedding customs for the selected religion.
    -
    Output format:
    1. [Ceremony Name] – [Date] at [Time]
       Notes: [Short description less than 30 words]

    """
    response = llm.invoke(prompt)
    text = response.content.strip()
    session["schedule_text"] = text
    save_wedding_session(session)
    return text

def update_schedule(existing_schedule, update_request):
    prompt = f"""
    Here is the current wedding schedule:

    {existing_schedule}

    User request: {update_request}

    Update the schedule accordingly.
    Maintain the same format and ensure timing/logical flow makes sense.
    """
    response = llm.invoke(prompt)
    return response.content.strip()

# -------------------------
# Budget Logic
# -------------------------
def allocate_budget(total_budget, guests, num_days, include_clothing=True):
    total = float(total_budget)
    base = {
        "Venue": 0.30, "Catering": 0.35, "Photographer": 0.10,
        "Music Band": 0.08, "Decorator": 0.07, "Clothing": 0.10
    }
    if not include_clothing:
        c_share = base.pop("Clothing")
        base["Catering"] += c_share * 0.5
        rem = c_share * 0.5
        for k in ["Venue", "Photographer", "Music Band", "Decorator"]:
            base[k] += rem * (base[k] / sum(base.values()))

    alloc = {k: round(v * total) for k, v in base.items()}
    diff = int(total) - sum(alloc.values())
    if diff:
        alloc["Catering"] += diff
    num_days = max(1, num_days)
    return {
        "allocations": alloc,
        "venue_per_day_target": round(alloc["Venue"] / num_days),
        "catering_per_plate_target": round(alloc["Catering"] / (guests + 100))
    }

def nearest_by_price(df, price_col, target, top_n=3):
    if df.empty or price_col not in df.columns:
        return pd.DataFrame([{"Note": "No data found"}])
    df2 = df.copy()
    df2[price_col] = pd.to_numeric(df2[price_col], errors="coerce").fillna(np.inf)
    df2["diff"] = (df2[price_col] - target).abs()
    return df2.sort_values("diff").head(top_n).drop(columns=["diff"])

def recommend_vendors(alloc_meta, num_days, guests, location):
    if not location or location.strip().lower() != "lucknow":
        msg = pd.DataFrame([{"Note": "Vendor data available only for Lucknow."}])
        return {"Venues": msg, "Caterers": msg, "Photographers": msg, "Music Bands": msg, "Decorators": msg}

    return {
        "Venues": nearest_by_price(venues_df, "Price", alloc_meta["venue_per_day_target"], 3),
        "Caterers": nearest_by_price(caterers_df, "Price", alloc_meta["catering_per_plate_target"], 5),
        "Photographers": nearest_by_price(photographers_df, "Price", alloc_meta["allocations"]["Photographer"], 3),
        "Music Bands": nearest_by_price(music_df, "Price", alloc_meta["allocations"]["Music Band"], 3),
        "Decorators": nearest_by_price(decorators_df, "Price", alloc_meta["allocations"]["Decorator"], 3)
    }

def plot_allocation_pie(alloc_meta):
    labels = list(alloc_meta["allocations"].keys())
    sizes = list(alloc_meta["allocations"].values())
    fig, ax = plt.subplots(figsize=(6,6))
    ax.pie(sizes, labels=labels, autopct=lambda p: f"₹{int(p/100*sum(sizes))}", startangle=140)
    ax.axis("equal")
    return fig

# -------------------------
# Full Flow
# -------------------------
def run_full_and_generate_all(name, religion, start_date, end_date, budget, location, ceremonies, guests, clothing):
    schedule = plan_with_groq(name, religion, start_date, end_date, budget, location, ceremonies, guests, clothing)
    try:
        sd = datetime.strptime(start_date, "%Y-%m-%d")
        ed = datetime.strptime(end_date, "%Y-%m-%d")
        num_days = (ed - sd).days + 1
    except:
        num_days = 1
    alloc_meta = allocate_budget(budget, guests, num_days, clothing)
    rec = recommend_vendors(alloc_meta, num_days, guests, location)
    alloc_df = pd.DataFrame(list(alloc_meta["allocations"].items()), columns=["Category", "Allocated (₹)"])
    fig = plot_allocation_pie(alloc_meta)
    return schedule, alloc_df, rec["Venues"], rec["Caterers"], rec["Photographers"], rec["Music Bands"], rec["Decorators"], fig

# -------------------------
# UI
# -------------------------
sess = load_wedding_session()

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 💒 ShaadiVerse — Planner + Budget Dashboard")

    with gr.Tabs():
        with gr.TabItem("📅 Planner"):
            with gr.Row():
                name = gr.Textbox(label="Couple Name", value=sess.get("name", ""))
                religion = gr.Dropdown(["Hindu","Sikh","Muslim","Christian","Other"], value=sess.get("religion","Hindu"))
            with gr.Row():
                start_date = gr.Textbox(label="Start Date (YYYY-MM-DD)", value=sess.get("start_date", ""))
                end_date = gr.Textbox(label="End Date (YYYY-MM-DD)", value=sess.get("end_date", ""))
            with gr.Row():
                budget = gr.Number(label="Total Budget (INR)", value=sess.get("budget", 1500000))
                guests = gr.Number(label="Guests", value=sess.get("guests", 300))
                clothing = gr.Checkbox(label="Include Clothing?", value=sess.get("clothing", True))
            location = gr.Textbox(label="Location", value=sess.get("location", "Lucknow"))
            ceremonies = gr.Textbox(label="Ceremonies", value=sess.get("ceremonies", "Haldi, Mehendi, Sangeet, Wedding, Reception"))
            gen_btn = gr.Button("Generate Schedule (AI)")
            schedule_out = gr.Textbox(label="Generated Schedule", lines=12)
            update_req = gr.Textbox(label="Update Request")
            update_btn = gr.Button("Update Schedule")
            gen_btn.click(plan_with_groq, [name, religion, start_date, end_date, budget, location, ceremonies, guests, clothing], schedule_out)
            update_btn.click(update_schedule, [schedule_out, update_req], schedule_out)

        with gr.TabItem("💰 Budget Optimizer"):
            run_all_btn = gr.Button("Run All (Planner + Budget)")
            alloc_table = gr.Dataframe(label="Budget Allocation")
            venue_table = gr.Dataframe(label="Venues")
            caterer_table = gr.Dataframe(label="Caterers")
            photog_table = gr.Dataframe(label="Photographers")
            music_table = gr.Dataframe(label="Music Bands")
            decor_table = gr.Dataframe(label="Decorators")
            chart = gr.Plot(label="Budget Distribution")

            run_all_btn.click(
                run_full_and_generate_all,
                [name, religion, start_date, end_date, budget, location, ceremonies, guests, clothing],
                [schedule_out, alloc_table, venue_table, caterer_table, photog_table, music_table, decor_table, chart]
            )

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://edfc4cba8ce4d26bda.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://edfc4cba8ce4d26bda.gradio.live
